In [64]:
#!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.23
    Uninstalling Cython-0.29.23:
      Successfully uninstalled Cython-0.29.23


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, which is not installed.
fbprophet 0.7.1 requires setuptools-git>=1.2, which is not installed.


In [2]:
##Author: Hayden Gill
##Last Updated: 5/26/2021
##Purpose: Perform LDA on Reddit WallStreetBets data, as part of IST736 Final Project
import nltk
import numpy as np
import pandas as pd
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from nltk import WordNetLemmatizer, PorterStemmer

import temp_script
from gensim.corpora.dictionary import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
from gensim import matutils
from gensim.test import test_utils, utils
from gensim.test.utils import datapath

In [2]:
comments = pd.read_csv("wsb_comments_results.csv")
posts = pd.read_csv("wsb_post_results.csv")

Set comments & posts index so as to re-combine later - and select just whats needed

In [15]:
comments = comments.loc[comments['author'] != "AutoModerator",:]
comments.set_index(['id','parent_id'], inplace = True)
comments_min = pd.DataFrame(comments['body'])
print(comments_min)

                                                                 body
id      parent_id                                                    
gftdfol t1_gftdcb8                                            dipping
gftdfo4 t1_gftdc1s                                   nio and spce ftw
gftdfo3 t1_gftdbth                                               flat
gftdfnp t3_kcvkwx   lol what happened to the limit up everyones be...
gftdfmf t1_gftddng                                               jan 
...                                                               ...
gkdmw91 t1_gkceomw  lmaogot nights to sleep over itmight belittle ...
gkdmw6k t3_l3c0hj   squeeze aside are there price targets for gmes...
gkdmw1y t1_gkdloa8                                      td ameritrade
gkdmvxa t1_gkbyl29  same ive gotin savings but thinking of droppin...
gkdmvqi t1_gkdgfwc  imagine being so uneducated and ingnorant you ...

[9486843 rows x 1 columns]


Set posts index so as to re-combine later, and select whats needed

In [16]:
posts.set_index(['id'], inplace = True)
posts_min  = pd.DataFrame(posts['body'])
print(posts_min)

                                                     body
id                                                       
kcz0hf   am woke up five minutes ago made some instant...
kcz07r                                            removed
kcyyw4                                            removed
kcyyve                                            removed
kcyyo1   intro let me get soft with ya it can get ugly...
...                                                   ...
kl8wbp  i dont know how the fuck you guys became so po...
kl8tjy                                            removed
kl8m4y  saw the trends and tryingback test to see if h...
kl8f6l  large gamestop investor ryan cohen buys more s...
kl8f3t   trading plan daily routine mf premarket am lo...

[424107 rows x 1 columns]


Removing the "removed" bodies from both comments and posts, as well as any with a word length less than 5. "lets all invest in gamestop" is basically short enough

In [243]:
##Dev area for below block - delete when done
print(posts_min)

                                                     body
id                                                       
kcz0hf   am woke up five minutes ago made some instant...
kcz07r                                            removed
kcyyw4                                            removed
kcyyve                                            removed
kcyyo1   intro let me get soft with ya it can get ugly...
...                                                   ...
kl8wbp  i dont know how the fuck you guys became so po...
kl8tjy                                            removed
kl8m4y  saw the trends and tryingback test to see if h...
kl8f6l  large gamestop investor ryan cohen buys more s...
kl8f3t   trading plan daily routine mf premarket am lo...

[424107 rows x 1 columns]


In [17]:
temp_posts = posts_min.copy(deep = True)
temp_comments = comments_min.copy(deep = True)
##handle the posts
print("reducing posts:")
before_removed = len(temp_posts)
temp_posts = temp_posts.loc[temp_posts['body'] != "removed",:]
after_removed = len(temp_posts)
print(str(before_removed - after_removed) + " 'removed' dropped")

before_length = len(temp_posts)
posts_word_counts = temp_posts['body'].apply(lambda x: len(nltk.word_tokenize(str(x))))
temp_posts = temp_posts.loc[(posts_word_counts >= 5),:]
after_length = len(temp_posts)
print(str(before_length - after_length) + " shorter-than-5 posts dropped")

print("\n\nreducing comments:")
before_removed = len(temp_comments)
temp_comments = temp_comments.loc[temp_comments['body'] != "removed",:]
after_removed = len(temp_comments)
print(str(before_removed - after_removed) + " 'removed' dropped") 

before_length = len(temp_comments)
comments_word_counts = temp_comments['body'].apply(lambda x: len(nltk.word_tokenize(str(x))))
print("\ndone")

reducing posts:
327625 'removed' dropped
43055 shorter-than-5 posts dropped


reducing comments:
2068380 'removed' dropped

done


In [18]:
#print(comments_word_counts)
#print(len(temp_comments))
mask = comments_word_counts >=5
temp_comments = temp_comments.loc[mask,:]
print(str(len(mask) - len(temp_comments)) + " shorter-than-5 comments dropped")

2201204 shorter-than-5 comments dropped


In [3]:
comments_min = temp_comments
comments_min.to_csv("comments_rm_short_removed.csv")

posts_min = temp_posts
posts_min.to_csv("posts_rm_short_removed.csv")

NameError: name 'temp_comments' is not defined

In [3]:
comments = pd.read_csv("comments_rm_short_removed.csv")
posts = pd.read_csv("posts_rm_short_removed.csv")
comments.set_index(['id','parent_id'], inplace = True)
comments_min = pd.DataFrame(comments['body'])

posts.set_index(['id'], inplace = True)
posts_min  = pd.DataFrame(posts['body'])

With the short posts and removed posts eliminated, we can format the text with count vectorizer, and think about removing
the infrequent words, choices for tokenization, stemming, lemmatization

In [3]:
##function that can be extended later to improve LDA results with combinations of tokenization, frequency, spellchecks, stemming, and lemmatization
def count_vectorize(documents, doc_frequency_threshhold = 10, tokenizer = None, stemming_yes_no = False, lemming_yes_no = False, spelling_yes_no = False):
    #print("Hello")
    cv_transformer = CountVectorizer(input = 'content'
                                , lowercase = True
                                , tokenizer = simple_preprocess
                                , stop_words = 'english'
                                , ngram_range = (1,3)
                                , min_df = doc_frequency_threshhold
                                , max_df = 1.0
                                , binary = True)
    if stemming_yes_no:
        print("stem")
        stemmer = PorterStemmer()
        documents = pd.DataFrame([' '.join([stemmer.stem(word) for word in text.split(' ')]) for text in documents['body']], columns = ['content'])
    
    if lemming_yes_no:
        print("lem")
        lemmer = WordNetLemmatizer()
        documents = pd.DataFrame([' '.join([lemmer.lemmatize(word) for word in text.split(' ')]) for text in documents['body']], columns = ['content'])

    if spelling_yes_no:
        print("spell")
    #print(documents)
    #print(type(documents))
    doc_matrix = cv_transformer.fit_transform(documents['body'])
    vocab = cv_transformer.get_feature_names()
    print("\nThe produced Term/Document matrix vocab is of size: " + str(len(vocab)))
    #print("\n and includes words: ")
    #print(vocab)
    print(str((doc_matrix.sum()/(len(vocab)*len(documents)))*100) + "% Non-Sparse")
    return(doc_matrix, vocab, cv_transformer)

def model_lda(cvect_doc_fit, vocabulary, num_topics = 5, passes = 20, modeling_cores = 7):
    print("Fitting LDA...")
    word_mapping = dict([(i, s) for i, s in enumerate(vocabulary)])
    temp = LdaMulticore(matutils.Sparse2Corpus(cvect_doc_fit.T)
                        , num_topics = num_topics
                        , passes = passes
                        , workers = modeling_cores
                        , id2word = word_mapping)
    return temp

def print_topics(lda, vocab, n=10):
    topics = lda.show_topics()
    #for ti, topic in enumerate(topics):
        #print('topic %d: %s' % (ti, ' '.join('%s/%.2f' % (t[1], t[0]) for t in topic)))

Now perform LDA modeling on Posts

In [6]:
#posts_corpus_basic, vocabulary = count_vectorize(posts_min, doc_frequency_threshhold = 5) .1%
#posts_corpus_basic, vocabulary = count_vectorize(posts_min, doc_frequency_threshhold = 10) .15%
#posts_corpus_basic, vocabulary = count_vectorize(posts_min, doc_frequency_threshhold = 20)
#posts_corpus_basic, vocabulary = count_vectorize(posts_min, doc_frequency_threshhold = 50)
#posts_corpus_basic, vocabulary = count_vectorize(posts_min, doc_frequency_threshhold = 100) #99.18% sparse
#posts_corpus_basic, vocabulary = count_vectorize(posts_min, doc_frequency_threshhold = 200) #98.5% sparse
posts_corpus_basic, vocabulary, posts_cv = count_vectorize(posts_min, doc_frequency_threshhold = 500)# 96% sparse, acceptable for now


The produced Term/Document matrix vocab is of size: 653
2.5396219100006534% Non-Sparse


In [7]:
print(posts_corpus_basic[1,:])

  (0, 14)	1
  (0, 575)	1
  (0, 5)	1
  (0, 582)	1
  (0, 305)	1
  (0, 348)	1
  (0, 470)	1
  (0, 354)	1
  (0, 651)	1
  (0, 533)	1
  (0, 309)	1
  (0, 540)	1
  (0, 618)	1
  (0, 176)	1
  (0, 561)	1
  (0, 188)	1
  (0, 421)	1
  (0, 531)	1
  (0, 109)	1
  (0, 646)	1
  (0, 448)	1
  (0, 487)	1
  (0, 255)	1
  (0, 84)	1
  (0, 214)	1
  :	:
  (0, 370)	1
  (0, 300)	1
  (0, 201)	1
  (0, 413)	1
  (0, 149)	1
  (0, 217)	1
  (0, 594)	1
  (0, 505)	1
  (0, 569)	1
  (0, 353)	1
  (0, 200)	1
  (0, 293)	1
  (0, 67)	1
  (0, 77)	1
  (0, 424)	1
  (0, 257)	1
  (0, 524)	1
  (0, 152)	1
  (0, 127)	1
  (0, 372)	1
  (0, 335)	1
  (0, 144)	1
  (0, 131)	1
  (0, 534)	1
  (0, 339)	1


In [32]:
temp = posts_corpus_basic.todense()
#print(temp[0])
posts_corpus_tokenized = []
vocab_array = np.asarray(vocabulary).reshape(1, 653)
for doc in posts_corpus_basic.todense():
    one_doc = np.asarray(doc).astype(bool)
    posts_corpus_tokenized.append(vocab_array[one_doc])


In [182]:
LDA_model = model_lda(posts_corpus_basic, vocabulary, num_topics = 5, modeling_cores = 7)

Fitting LDA...


TypeError: show_topics() got an unexpected keyword argument 'topics'

In [188]:
#print_topics(LDA_model, vocabulary, n = 10)
LDA_model.show_topics()


[(0,
  '0.040*"hold" + 0.029*"shares" + 0.028*"sell" + 0.027*"buy" + 0.026*"gme" + 0.021*"dont" + 0.014*"squeeze" + 0.014*"just" + 0.014*"holding" + 0.013*"short"'),
 (1,
  '0.091*"gme" + 0.076*"amc" + 0.061*"buy" + 0.049*"bb" + 0.041*"lets" + 0.039*"nok" + 0.038*"moon" + 0.022*"robinhood" + 0.019*"just" + 0.018*"going"'),
 (2,
  '0.024*"im" + 0.016*"just" + 0.015*"money" + 0.014*"like" + 0.013*"know" + 0.011*"people" + 0.011*"ive" + 0.011*"make" + 0.010*"dont" + 0.010*"going"'),
 (3,
  '0.012*"company" + 0.011*"stock" + 0.011*"price" + 0.010*"market" + 0.009*"short" + 0.008*"shares" + 0.007*"going" + 0.007*"year" + 0.007*"time" + 0.007*"like"'),
 (4,
  '0.033*"robinhood" + 0.023*"market" + 0.022*"hedge" + 0.021*"funds" + 0.017*"trading" + 0.015*"hedge funds" + 0.012*"stocks" + 0.011*"people" + 0.011*"manipulation" + 0.011*"like"')]

In [24]:
from gensim.test.utils import datapath
temp_file = datapath("posts_model_1")
LDA_model.save(temp_file)
LDA_post_model = LDA_model

NameError: name 'LDA_model' is not defined

In [4]:
##try to load the model already made to examine in pycharm
from gensim.test.utils import datapath
temp_file = datapath("posts_model_1")
LDA_post_model = LdaMulticore.load(temp_file)
print("success")
#LDA_post_model.show_topics()

success


In [57]:
d = corpora.Dictionary()
d.id2token = LDA_post_model.id2word
d.token2id = dict((k, v) for k, v in posts_cv.vocabulary_.items())
post_dictionary = d
##coherence model requires the original text - which has been lost due to count vectorizer

coherence_model_lda = CoherenceModel(model=LDA_post_model, texts=posts_corpus_tokenized, dictionary = d)
coherence_model_lda.get_coherence()
#.69 coherence is pretty good


0.6911339380472198

LDA for Comments

In [5]:
#comments_corpus_basic, comments_vocabulary = count_vectorize(comments_min, doc_frequency_threshhold = 5) # essentially all sparse
#comments_corpus_basic, comments_vocabulary = count_vectorize(comments_min, doc_frequency_threshhold = 500)# 99.998% sparse, acceptable for now
#print("5k")
#comments_corpus_basic, comments_vocabulary = count_vectorize(comments_min, doc_frequency_threshhold = 5000)#99.98% sparse, 1600 words
print("15k")
comments_corpus_basic, comments_vocabulary, comments_cv = count_vectorize(comments_min, doc_frequency_threshhold = 300)#99.97% sparse, 600 words
##will stick with 602 because comments will be at least as complex as posts

15k

The produced Term/Document matrix vocab is of size: 20383
0.04743504782624755% Non-Sparse


In [42]:
#temp = comments_corpus_basic.todense()
#print(temp[0])
comments_corpus_tokenized = []
vocab_array = np.asarray(comments_vocabulary).reshape(1, 20383)
for doc in comments_corpus_basic:
    doc = doc.todense()
    one_doc = np.asarray(doc).astype(bool)
    comments_corpus_tokenized.append(vocab_array[one_doc])

In [10]:
LDA_comment_model = model_lda(comments_corpus_basic, comments_vocabulary, num_topics = 10, modeling_cores = 7)

Fitting LDA...


KeyboardInterrupt: 

In [13]:
LDA_comment_model.show_topics()

[(0,
  '0.047*"im" + 0.024*"fucking" + 0.023*"holding" + 0.017*"calls" + 0.014*"pltr" + 0.013*"today" + 0.012*"gme" + 0.012*"bought" + 0.010*"moon" + 0.010*"lets"'),
 (1,
  '0.049*"like" + 0.020*"just" + 0.019*"hands" + 0.017*"retarded" + 0.016*"days" + 0.015*"wait" + 0.014*"ago" + 0.013*"whats" + 0.013*"diamond" + 0.012*"look"'),
 (2,
  '0.026*"make" + 0.025*"money" + 0.024*"good" + 0.021*"going" + 0.016*"im" + 0.016*"like" + 0.011*"sure" + 0.008*"year" + 0.007*"feel" + 0.007*"bad"'),
 (3,
  '0.019*"stock" + 0.013*"short" + 0.010*"market" + 0.010*"long" + 0.009*"like" + 0.008*"just" + 0.007*"company" + 0.007*"price" + 0.007*"doesnt" + 0.004*"term"'),
 (4,
  '0.073*"buy" + 0.059*"gme" + 0.054*"hold" + 0.037*"shares" + 0.034*"amc" + 0.024*"dip" + 0.022*"buying" + 0.020*"just" + 0.018*"bought" + 0.017*"bb"'),
 (5,
  '0.053*"just" + 0.049*"dont" + 0.031*"know" + 0.023*"youre" + 0.014*"right" + 0.012*"want" + 0.010*"im" + 0.009*"like" + 0.008*"read" + 0.008*"advice"'),
 (6,
  '0.045*"sell"

In [252]:
temp_file = datapath("comment_model_1")
LDA_comment_model.save(temp_file)

In [14]:
temp_file = datapath("comment_model_1")
LDA_comment_model = LdaMulticore.load(temp_file)
print("success")

success


In [15]:
LDA_comment_model.show_topics()

[(0,
  '0.137*"buy" + 0.087*"shares" + 0.056*"hold" + 0.041*"gme" + 0.037*"robinhood" + 0.035*"bought" + 0.034*"dip" + 0.032*"sell" + 0.031*"just" + 0.024*"account"'),
 (1,
  '0.088*"fuck" + 0.063*"thats" + 0.060*"shit" + 0.034*"let" + 0.034*"theyre" + 0.030*"yeah" + 0.026*"wont" + 0.023*"stop" + 0.022*"real" + 0.022*"time"'),
 (2,
  '0.144*"dont" + 0.077*"know" + 0.065*"got" + 0.051*"need" + 0.045*"stock" + 0.044*"want" + 0.043*"like" + 0.033*"probably" + 0.028*"just" + 0.024*"guy"'),
 (3,
  '0.056*"market" + 0.045*"calls" + 0.042*"price" + 0.030*"today" + 0.024*"day" + 0.023*"open" + 0.019*"shorts" + 0.018*"sell" + 0.018*"puts" + 0.018*"stock"'),
 (4,
  '0.070*"good" + 0.041*"does" + 0.037*"say" + 0.034*"man" + 0.032*"better" + 0.031*"doesnt" + 0.030*"come" + 0.029*"work" + 0.025*"thanks" + 0.025*"way"'),
 (5,
  '0.222*"im" + 0.056*"gonna" + 0.046*"think" + 0.043*"long" + 0.039*"just" + 0.037*"sure" + 0.028*"play" + 0.026*"ill" + 0.023*"looking" + 0.022*"pretty"'),
 (6,
  '0.134*"gme

In [16]:
d = corpora.Dictionary()
d.id2token = LDA_comment_model.id2word
d.token2id = dict((k, v) for k, v in comments_cv.vocabulary_.items())
comment_dictionary = d
##coherence model requires the original text - which has been lost due to count vectorizer

coherence_comment_model_lda = CoherenceModel(model=LDA_comment_model, texts=comments_corpus_tokenized, dictionary = d)
coherence_comment_model_lda.get_coherence()
#.54 coherence - could be better - but not terrible- improved to .58 with lemmatizing and stemming

KeyboardInterrupt: 

In [ ]:
d = corpora.Dictionary()
d.id2token = LDA_comment_model.id2word
d.token2id = dict((k, v) for k, v in comments_cv.vocabulary_.items())
comment_dictionary = d
def try_lda_options(comments_corpus_basic, comments_vocabulary, comments_corpus_tokenized, d, num_topics = num_topics):
    for topic_trial in num_topics:
        LDA_comment_model = model_lda(comments_corpus_basic, comments_vocabulary, num_topics = topic_trial, modeling_cores = 7)
        coherence_comment_model_lda = CoherenceModel(model=LDA_comment_model, texts=comments_corpus_tokenized, dictionary = d)
        coherence_comment_model_lda.get_coherence()

In [39]:
display(comments_min.index)

MultiIndex([('gftdfnp',  't3_kcvkwx'),
            ('gftdfk8',  't3_kcvkwx'),
            ('gftdfh0', 't1_gftd073'),
            ('gftdfek',  't3_kcvkwx'),
            ('gftdfb7',  't3_kcvkwx'),
            ('gftdf7b',  't3_kcvkwx'),
            ('gftdf4z', 't1_gftcijz'),
            ('gftdf1s',  't3_kcvkwx'),
            ('gftdf0t',  't3_kcvkwx'),
            ('gftdf0e',  't3_kcvkwx'),
            ...
            ('gkdmx3l', 't1_gkdh9q3'),
            ('gkdmx31', 't1_gkcx6n1'),
            ('gkdmx1g',  't3_l3aj4z'),
            ('gkdmwu8', 't1_gkb9hsw'),
            ('gkdmwoy',  't3_l35s4w'),
            ('gkdmwie', 't1_gkdmaa1'),
            ('gkdmw91', 't1_gkceomw'),
            ('gkdmw6k',  't3_l3c0hj'),
            ('gkdmvxa', 't1_gkbyl29'),
            ('gkdmvqi', 't1_gkdgfwc')],
           names=['id', 'parent_id'], length=5217259)

With both topic models created, final step is to print out the topics as well as create a mapping of topics to documents

In [30]:
#posts_bow = [post_dictionary.doc2bow(text) for text in posts_corpus_tokenized]
#topics_by_post = LDA_post_model[posts_bow]

comments_bow = [comment_dictionary.doc2bow(text) for text in comments_corpus_tokenized]

In [40]:
topics_by_comment = LDA_comment_model[comments_bow]
print("done")

done


In [41]:
print(pd.DataFrame(topics_by_comment).head)

IndexError: index 5197 is out of bounds for axis 1 with size 520

In [40]:
#comment_topics = pd.DataFrame(topics_by_comment)
#post_topics = pd.DataFrame(topics_by_post)

comment_topics['id'] = comments_min.index
#comment_topics['parent_id'] = comments_min['parent_id']

#post_topics[id] = posts_min['id']

comment_topics.to_csv("comments_with_topics.csv")
#post_topics.to_csv("posts_with_topics.csv")

#comment_topic_specs = LDA_comment_model

pd.DataFrame(LDA_comment_model.show_topics()).to_csv("WSB_Comment_Topics.csv")

In [19]:
pd.DataFrame(LDA_comment_model.show_topics()).to_csv("WSB_Comment_Topics.csv")
#pd.DataFrame(LDA_post_model.show_topics()).to_csv("WSB_Post_Topics.csv")

With the models created and saved, lets see how meaningful they are

In [19]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [36]:
#This nice function above requires the gensim bag of words representation. We'll give it just that'
comments_small = pd.DataFrame(comments[:10])
corpus_small = comments_corpus_tokenized[:10]
##doc to bag of words ONLY WORKS WITH AN ARRAY OF WORDS
corpus_bow_small = [comment_dictionary.doc2bow(text) for text in corpus_small]
print(LDA_comment_model[corpus_bow_small])

In [37]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=LDA_comment_model, corpus=comments_bow, texts=comments_corpus_tokenized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

IndexError: index 5197 is out of bounds for axis 1 with size 520

In [38]:
for i, row in enumerate(LDA_comment_model[comments_bow]):
    print(i)
    print(row)

IndexError: index 5197 is out of bounds for axis 1 with size 520